In [12]:
import cv2 
import os.path as osp
import os
import glob
from mmdet.apis import inference_detector, init_detector
from mmpose.apis import inference_top_down_pose_model, init_pose_model,process_mmdet_results,vis_pose_result
import mmcv
import numpy as np
import shutil


In [13]:
def extract_frame(video_path):
    dname = 'temp'
    os.makedirs(dname, exist_ok=True)
    frame_tmpl = osp.join(dname, 'img_{:05d}.jpg')
    cap = cv2.VideoCapture(video_path)
    frame_paths = []
    cnt = 0
    while(cap.isOpened()):
        flag, frame = cap.read()
        if flag:
            frame_path = frame_tmpl.format(cnt + 1)
            frame_paths.append(frame_path)
            cv2.imwrite(frame_path, frame)
            cnt += 1
        else: break
    cap.release()
    return frame_paths

# pose_config = 'mmpose/configs/body/2d_kpt_sview_rgb_img/topdown_heatmap/coco/hrnet_w48_coco_256x192.py'
# pose_checkpoint = 'hrnet_w48_coco_256x192-b9e0b3ab_20200708.pth'
# ###########
# det_config = 'Pose/yolox_s_8x8_300e_coco.py'
# det_checkpoint = 'Pose/yolox_s_8x8_300e_coco_20211121_095711-4592a793.pth'
###############
pose_config = 'Pose/hrnet_w48_coco_256x192.py'
pose_checkpoint = 'Pose/hrnet_w48_coco_256x192-b9e0b3ab_20200708.pth'
###########
det_config = 'Pose/yolox_s_8x8_300e_coco.py'
det_checkpoint = 'Pose/yolox_s_8x8_300e_coco_20211121_095711-4592a793.pth'
##############
# initialize pose model
pose_model = init_pose_model(pose_config, pose_checkpoint)
# initialize detector
det_model = init_detector(det_config, det_checkpoint)
def detection_inference(det_config, det_checkpoint ,frame_paths, det_score_thr=0.5,device='cuda' ):
    model = init_detector(det_config, det_checkpoint, device)
    assert model.CLASSES[0] == 'person', ('We require you to use a detector '
                                          'trained on COCO')
    results = []
    print('Performing Human Detection for each frame')
    prog_bar = mmcv.ProgressBar(len(frame_paths))
    for frame_path in frame_paths:
        result = inference_detector(model, frame_path)
        # We only keep human detections with score larger than det_score_thr
        result = result[0][result[0][:, 4] >= det_score_thr]
        results.append(result)
        prog_bar.update()
    return results

def pose_inference(pose_config,pose_checkpoint, frame_paths,image_shape, det_results, device='cuda'):
    model = init_pose_model(pose_config, pose_checkpoint, device)
    print('Performing Human Pose Estimation for each frame')
    prog_bar = mmcv.ProgressBar(len(frame_paths))

    num_frame = len(det_results)
    num_person = max([len(x) for x in det_results])
    kp = np.zeros((num_person, num_frame, 17, 3), dtype=np.float32)
    for i, (f, d) in enumerate(zip(frame_paths, det_results)):
        # Align input format
        # if len(d) == 0: 
        #     prog_bar.update()
        #     continue
        d = [dict(bbox=x) for x in list(d) if x[-1] > 0.5]
        pose = inference_top_down_pose_model(model, f, d, format='xyxy')[0]
        vis_ske = vis_pose_result(model,f,pose,dataset=model.cfg.data.test.type,show=False)
        if cv2.waitKey(20)& 0xFF==ord('q'): break
        cv2.imshow('',vis_ske)
        for j, item in enumerate(pose):
            normkp = item['keypoints']/(image_shape + (1,))
            kp[j, i] = normkp
        prog_bar.update()
    cv2.destroyAllWindows()
    return kp

def pose_extraction(vid,det_config, det_checkpoint,pose_config,pose_checkpoint,label, det_score_thr=0.5,device='cuda'):
    frame_paths = extract_frame(vid)
    det_results = detection_inference(det_config, det_checkpoint ,frame_paths, det_score_thr,device)
    image = cv2.imread(frame_paths[0])
    image_shape = (image.shape[1], image.shape[0])
    pose_results = pose_inference(pose_config,pose_checkpoint, frame_paths,image_shape, det_results, device)
    anno = dict()
    anno['kp'] = pose_results[..., :2]
    anno['kp_score'] = pose_results[..., 2]
    anno['frame_dir'] = osp.splitext(osp.basename(vid))[0]
    anno['img_shape'] = image_shape
    anno['original_shape'] = image_shape
    anno['total_frames'] = pose_results.shape[1]
    anno['label'] = label
    shutil.rmtree(osp.dirname(frame_paths[0]))
    return anno


load checkpoint from local path: Pose/hrnet_w48_coco_256x192-b9e0b3ab_20200708.pth
load checkpoint from local path: Pose/yolox_s_8x8_300e_coco_20211121_095711-4592a793.pth


In [16]:

# files=glob.glob('VIDEO_TEST/*.mp4')
# file='VIDEO_TEST/VIDEO11.mp4'
lstfile = []
lstfile.append(sorted(glob.glob('Video_running/*.mp4'),key=os.path.getmtime))
lstfile.append(sorted(glob.glob('Video_Fall1/*.mp4'),key=os.path.getmtime))
lstfile.append(sorted(glob.glob('Video_Lying/*.mp4'),key=os.path.getmtime))
# lstfile=sorted(key=os.path.getmtime)
anno_train = []
for index,files in enumerate(lstfile):
    for file in files:
        print('Processing ' + file)
        anno = pose_extraction(file, det_config, det_checkpoint,pose_config,pose_checkpoint,label=index) #LABEL NOT_FALL
        anno_train.append(anno)

Processing Video_running\Video1.mp4
load checkpoint from local path: Pose/yolox_s_8x8_300e_coco_20211121_095711-4592a793.pth
Performing Human Detection for each frame
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 292/292, 18.6 task/s, elapsed: 16s, ETA:     0sload checkpoint from local path: Pose/hrnet_w48_coco_256x192-b9e0b3ab_20200708.pth
Performing Human Pose Estimation for each frame
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 292/292, 5.4 task/s, elapsed: 54s, ETA:     0sProcessing Video_running\Video2.mp4
load checkpoint from local path: Pose/yolox_s_8x8_300e_coco_20211121_095711-4592a793.pth
Performing Human Detection for each frame
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 73/73, 17.7 task/s, elapsed: 4s, ETA:     0sload checkpoint from local path: Pose/hrnet_w48_coco_256x192-b9e0b3ab_20200708.pth
Performing Human Pose Estimation for each frame
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 73/73, 5.4 task/s, elapsed: 14s, ETA:     0sProcessing Video_running\Video3.mp4
load checkpoint from local path: Pose/yolox_s_8x8_30

In [20]:
file_test='VIDEO_TEST/VIDEO3.mp4'
print('Processing ' + file_test)
anno = pose_extraction(file_test, det_config, det_checkpoint,pose_config,pose_checkpoint,label=index) #LABEL NOT_FALL
# anno_train.append(anno)

Processing VIDEO_TEST/VIDEO3.mp4
load checkpoint from local path: Pose/yolox_s_8x8_300e_coco_20211121_095711-4592a793.pth
Performing Human Detection for each frame
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 609/609, 26.6 task/s, elapsed: 23s, ETA:     0sload checkpoint from local path: Pose/hrnet_w48_coco_256x192-b9e0b3ab_20200708.pth
Performing Human Pose Estimation for each frame
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 609/609, 10.5 task/s, elapsed: 58s, ETA:     0s

In [4]:
# for frame in frame_paths:
#     img = cv2.imread(frame)
#     if cv2.waitKey(20)& 0xFF==ord('q'): break
#     cv2.imshow('',img)
# cv2.destroyAllWindows()

In [5]:
# model = init_pose_model(pose_config, pose_checkpoint, 'cuda')
# print('Performing Human Pose Estimation for each frame')
# prog_bar = mmcv.ProgressBar(len(frame_paths))

# num_frame = len(results)
# num_person = max([len(x) for x in results])
# kp = np.zeros((num_person, num_frame, 17, 3), dtype=np.float32)
# for i, (f, d) in enumerate(zip(frame_paths, results)):
#     # Align input format
#     d = [dict(bbox=x) for x in list(d) if x[-1] > 0.5]
#     pose = inference_top_down_pose_model(model, f, d, format='xyxy')[0]
#     for j, item in enumerate(pose):
#         kp[j, i] = item['keypoints']
#     prog_bar.update()


In [7]:
# # model = init_detector(det_config, det_checkpoint)
# # assert model.CLASSES[0] == 'person', ('We require you to use a detector '
# #                                         'trained on COCO')
# # results = []
# # print('Performing Human Detection for each frame')
# # prog_bar = mmcv.ProgressBar(len(frame_paths))
# # det_score_thr=0.5
# # for frame_path in frame_paths:
# #     result_det = inference_detector(model, frame_path)
# #     # We only keep human detections with score larger than det_score_thr
# #     result = result_det[0][result_det[0][:, 4] >= det_score_thr]
# #     results.append(result)
# #     prog_bar.update()
# ashape=(2,2)
# bshape=ashape+(1,)
# bshape

In [8]:
# image = cv2.imread(frame_paths[0])
# image_shape = (image.shape[0], image.shape[1])
# pose_results = pose_inference(pose_config,pose_checkpoint, frame_paths, results)

In [9]:
# pose_results[0,5]

In [10]:
# a=pose_results[...,:2]
# a[0,5]

In [ ]:
anno['kp_score'][0,5]

In [ ]:
anno['kp'][0,5]

In [17]:
mmcv.dump(anno_train, 'train.pkl')
import pandas as pds
data=pds.read_pickle('train.pkl')

62

In [ ]:
data[0]

In [ ]:
type('train.pkl')